In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GaussianNoise, GlobalMaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [3]:
#Downloading dataset
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

 96% 165M/172M [00:01<00:00, 158MB/s]
100% 172M/172M [00:01<00:00, 145MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
#Loading the images
x_train= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

x_test= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.


In [6]:
class_names = x_train.class_names
class_names

['fractured', 'not fractured']

In [7]:
# Normalizing
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

x_train = x_train.map(process)
x_test = x_test.map(process)

In [8]:
input = (256, 256, 3)

In [10]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [11]:
base_model = MobileNetV2(weights='imagenet', include_top=False)

9406464/9406464 [==============================] - 0s 0us/step


In [12]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [13]:
for layer in base_model.layers[:100]:
    layer.trainable = False

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, None, None, 32)       864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, None, None, 32)       128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, None, None, 32)       0         ['bn_Conv1[0][0]']        

In [15]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [16]:
history = model.fit(x_train,epochs=10, batch_size=100, validation_data=x_test)

Epoch 1/10
89/89 [==============================] - 50s 278ms/step - loss: 0.1689 - accuracy: 0.9372 - val_loss: 2.2437 - val_accuracy: 0.6500
Epoch 2/10
89/89 [==============================] - 24s 257ms/step - loss: 0.0261 - accuracy: 0.9907 - val_loss: 4.2158 - val_accuracy: 0.6750
Epoch 3/10
89/89 [==============================] - 25s 263ms/step - loss: 0.0162 - accuracy: 0.9939 - val_loss: 4.2030 - val_accuracy: 0.7350
Epoch 4/10
89/89 [==============================] - 25s 270ms/step - loss: 0.0068 - accuracy: 0.9979 - val_loss: 21.4348 - val_accuracy: 0.4817
Epoch 5/10
89/89 [==============================] - 25s 269ms/step - loss: 0.0115 - accuracy: 0.9962 - val_loss: 27.2784 - val_accuracy: 0.4000
Epoch 6/10
89/89 [==============================] - 24s 262ms/step - loss: 0.0214 - accuracy: 0.9941 - val_loss: 3.0354 - val_accuracy: 0.6933
Epoch 7/10
89/89 [==============================] - 25s 272ms/step - loss: 0.0119 - accuracy: 0.9973 - val_loss: 3.4897 - val_accuracy: 0.64